## Инструменты
- Python
- приближённый KNN (nmslib/faiss/scann)
- модели из Hugging Face (Transformers)

[Рейтинг русскоязычных энкодеров предложений](https://habr.com/ru/articles/669674/)

In [5]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
# нужно еще это тут
# pip install ipywidgets
import torch
import nmslib
# работает на версии python 3.8.17
# https://stackoverflow.com/questions/71242919/pip-install-results-in-this-error-cl-exe-failed-with-exit-code-2

## Цель
- реализовать задачу классификации на основе BERT-like модели и KNN на данных [Russian Intents Dataset](https://www.kaggle.com/datasets/constantinwerner/qa-intents-dataset-university-domain) с Kaggle.
- научиться создавать классификаторы текстов в условиях большого числа маленьких классов, состоящих из коротких текстов

## Ожидаемый результат
- код для создания поискового векторного индекса
- логика определения класса на основе близости к обучающим объектам (по ближайшему, по топ-N ближайших, и т. п.).

## Dataset

In [6]:
train = pd.read_table(
    'qa-intents-dataset-university-domain/dataset_train.tsv',
    header=None,names=['txt','c'])

In [7]:
train

,txt,c
0,мне нужна справка,statement_general
1,оформить справку,statement_general
2,взять справку,statement_general
3,справку как получить,statement_general
4,справку ммф где получаться,statement_general
...,...,...
13225,тупой,smalltalk_abuse
13226,робот бестолковый,smalltalk_abuse
13227,несообразительный,smalltalk_abuse
13228,ты бестолковый,smalltalk_abuse


In [8]:
test = pd.read_table(
    'qa-intents-dataset-university-domain/dataset_test.tsv',
    header=None,names=['txt','c'])

## Tokenizer

In [11]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output['last_hidden_state']
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

## KNN
### код для создания поискового векторного индекса

In [90]:
def get_embeding(MODEL_NAME, train, test):
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModel.from_pretrained(MODEL_NAME)

    train_token = tokenizer(train['txt'].tolist(), padding=True, return_tensors='pt', max_length = 32, truncation=True) 
    test_token = tokenizer(test['txt'].tolist(), padding=True, return_tensors='pt', max_length = 32, truncation=True) 

    with torch.no_grad():
        train_embeding = mean_pooling(model(**train_token), train_token['attention_mask'])
        test_embeding = mean_pooling(model(**test_token), test_token['attention_mask'])
    return train_embeding, test_embeding

In [86]:
def get_index(train_embeding):
    index = nmslib.init(method='hnsw', space='cosinesimil')
    index.addDataPointBatch(train_embeding, ids=list(range(len(train_embeding))))
    index.createIndex({'post': 2}, print_progress=True)
    return index

### логика определения класса на основе близости к обучающим объектам, а именно по ближайшему

In [88]:
def get_result(index, test_embeding, test, train):

    right = 0
    # кол-во совпадений

    for n, test_item in enumerate(test_embeding):
        target = test.iloc[n].c
        predicted = None
        id_, dist = index.knnQuery(test_item, k=1)
        dict_ = {}
        for i, d in zip(id_, dist):
            exist_ = 0
            if train.iloc[i].c in dict_.keys():
                exist_ = dict_[train.iloc[i].c]
            dict_[train.iloc[i].c] = exist_ + 1 - d
        predicted = max(dict_, key=dict_.get)

        if target == predicted:
            right += 1
        else:
            pass
        
    return right / len(test_embeding)

## sentence-transformers/paraphrase-xlm-r-multilingual-v1

In [91]:
MODEL_NAME = 'sentence-transformers/paraphrase-xlm-r-multilingual-v1'
train_embeding, test_embeding = get_embeding(MODEL_NAME, train, test)

In [92]:
index = get_index(train_embeding)
result = get_result(index, test_embeding, test, train)
result

0.8856172140430351

## sentence-transformers/all-MiniLM-L6-v2

In [93]:
MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'
train_embeding, test_embeding = get_embeding(MODEL_NAME, train, test)

In [94]:
index = get_index(train_embeding)
result = get_result(index, test_embeding, test, train)
result

0.9003397508493771

## intfloat/multilingual-e5-large

In [62]:
MODEL_NAME = 'intfloat/multilingual-e5-large'
train_embeding, test_embeding = get_embeding(MODEL_NAME, train, test)

In [64]:
index = get_index(train_embeding)
result = get_result(index, test_embeding, test, train)
result

0.942242355605889


## inkoziev/sbert_synonymy

In [65]:
MODEL_NAME = 'inkoziev/sbert_synonymy'
train_embeding, test_embeding = get_embeding(MODEL_NAME, train, test)

In [67]:
index = get_index(train_embeding)
result = get_result(index, test_embeding, test, train)
result

0.9161947904869762


## cointegrated/rubert-tiny2

In [95]:
MODEL_NAME = 'cointegrated/rubert-tiny2'
train_embeding, test_embeding = get_embeding(MODEL_NAME, train, test)

In [96]:
index = get_index(train_embeding)
result = get_result(index, test_embeding, test, train)
result

0.8844847112117781

## ai-forever/sbert_large_nlu_ru

In [72]:
MODEL_NAME = 'ai-forever/sbert_large_nlu_ru'
train_embeding, test_embeding = get_embeding(MODEL_NAME, train, test)

In [75]:
index = get_index(train_embeding)
result = get_result(index, test_embeding, test, train)
result

0.8573046432616082


## xlm-roberta-base

In [76]:
MODEL_NAME = 'xlm-roberta-base'
train_embeding, test_embeding = get_embeding(MODEL_NAME, train, test)

In [78]:
index = get_index(train_embeding)
result = get_result(index, test_embeding, test, train)
result

0.8731596828992072


## DeepPavlov/rubert-base-cased-sentence

In [79]:
MODEL_NAME = 'DeepPavlov/rubert-base-cased-sentence'
train_embeding, test_embeding = get_embeding(MODEL_NAME, train, test)

In [81]:
index = get_index(train_embeding)
result = get_result(index, test_embeding, test, train)
result

0.8958097395243488


## bert-base-multilingual-cased

In [83]:
from transformers import BertTokenizer, BertModel

MODEL_NAME = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertModel.from_pretrained(MODEL_NAME)

train_token = tokenizer(train['txt'].tolist(), padding=True, return_tensors='pt', max_length = 32, truncation=True) 
test_token = tokenizer(test['txt'].tolist(), padding=True, return_tensors='pt', max_length = 32, truncation=True) 

with torch.no_grad():
    train_embeding = mean_pooling(model(**train_token), train_token['attention_mask'])
    test_embeding = mean_pooling(model(**test_token), test_token['attention_mask'])

In [85]:
index = get_index(train_embeding)
result = get_result(index, test_embeding, test, train)
result

0.8720271800679502


## Итог
- базовые модели (bert-base-multilingual-cased, xlm-roberta-base) дают средний результат
- модели предназначенные чисто под русский язык (ai-forever/sbert_large_nlu_ru, cointegrated/rubert-tiny2) тоже не самые лучшие
- хороший результат дала авторская модель (inkoziev/sbert_synonymy) специально сделаная для сравнения похожести русских предложений
- и очень хороший результат (точность 94%) дала модель с большим объемом парамметров (intfloat/multilingual-e5-large)